## test

In [ ]:
import os

import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

## import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D, Flatten
from tensorflow.keras.layers import AveragePooling2D, BatchNormalization, Conv2DTranspose
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.utils import to_categorical
from livelossplot.inputs.tf_keras import PlotLossesCallback

## functions

In [ ]:
sub = []
my_subjects = {}
counter = 0

files = os.listdir('F:/poorya/datasets/keypoint J dataset/pretext dataset/train_aug/')

for i in files:
    sub.append(i[-6:-4])

for i in range(95):
    if str(i).zfill(2) in sub:
        my_subjects[str(i).zfill(2)] = counter
        counter = counter + 1

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, folder, batch_size, my_subjects):
        self.folder = folder
        self.n = os.listdir(folder)
        self.batch_size = batch_size
        self.my_subjects = my_subjects
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.n) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        list_IDs_temp = [self.n[k] for k in indexes]
        x, y = self.__data_generation(list_IDs_temp)
        return x, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.n))

    def __data_generation(self, list_IDs_temp):
        images = np.empty((self.batch_size, 384, 384, 1))
        masks = np.empty((self.batch_size, 48, 48, 1))
        rotation = np.empty((self.batch_size), dtype = int)
        subject = np.empty((self.batch_size), dtype = int)
        gender = np.empty((self.batch_size), dtype = int)

        for i, ID in enumerate(list_IDs_temp):

            train_img_1 = image.img_to_array(image.load_img(self.folder + ID, color_mode = "grayscale", target_size = (
                384, 384)))
            train_img_1 /= 255.0
            masks[i,] = train_img_1[168:216, 168:216]

            train_img_2 = image.img_to_array(image.load_img(self.folder + ID, color_mode = "grayscale", target_size = (
                384, 384)))
            train_img_2 /= 255.0
            train_img_2[168:216, 168:216] = 0
            images[i,] = train_img_2

            rotation[i] = int(ID[-10])
            subject[i] = int(self.my_subjects[ID[-6:-4]])
            gender[i] = int(ID[-8])

        return images, [to_categorical(gender, num_classes = 2), to_categorical(rotation, num_classes = 4),
                        to_categorical(subject, num_classes = 89), masks]


bs = 16
train_gen = DataGenerator('F:/poorya/datasets/keypoint J dataset/pretext dataset/train_aug/', bs, my_subjects)
val_gen = DataGenerator('F:/poorya/datasets/keypoint J dataset/pretext dataset/val/', bs, my_subjects)
test_gen = DataGenerator('F:/poorya/datasets/keypoint J dataset/pretext dataset/test/', bs, my_subjects)

## model structure

In [ ]:
def vgg_block(layer_in, n_filters, n_conv):
    for _ in range(n_conv):
        layer_in = Conv2D(n_filters, (3, 3), padding = 'same', activation = 'relu')(layer_in)
    layer_in = BatchNormalization()(layer_in)
    layer_in = AveragePooling2D((2, 2), strides = (2, 2))(layer_in)
    return layer_in


def PSNR(super_resolution, high_resolution):
    psnr_value = tf.image.psnr(high_resolution, super_resolution, max_val = 1)[0]
    return psnr_value


def SSIMLoss(y_true, y_pred):
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))


def get_model():

    ######################################  INITIALIZATION  ########################################

    loss_list = {'gender': 'categorical_crossentropy', 'rotation': 'categorical_crossentropy', 'subject': 'categorical_crossentropy', 'mask': 'mse'}
    test_metrics = {'gender': 'accuracy', 'rotation': 'accuracy', 'subject': 'accuracy', 'mask': PSNR}
    opt = tf.keras.optimizers.RMSprop(learning_rate = 0.01)
    drp = 0.1
    act = 'relu'
    eact = 'softmax'
    s = 'same'

    ########################################  BACKBONE  ############################################

    model_input = Input(shape = (384, 384, 1))

    layer = vgg_block(model_input, 8, 2)
    layer = vgg_block(layer, 16, 2)
    layer = vgg_block(layer, 32, 2)
    layer = vgg_block(layer, 64, 2)
    layer = vgg_block(layer, 128, 2)
    layer = vgg_block(layer, 256, 2)

    ###################################  CLASSIFICATIONS  ##########################################

    x = Flatten()(layer)
    x = Dense(512, activation = act)(x)
    x = Dropout(drp)(x)

    y1 = Dense(256, activation = act)(x)
    y1 = Dropout(drp)(y1)
    y1 = Dense(128, activation = act)(y1)
    y1 = Dropout(drp)(y1)
    y1 = Dense(2, activation = eact, name = 'gender')(y1)

    y2 = Dense(256, activation = act)(x)
    y2 = Dropout(drp)(y2)
    y2 = Dense(128, activation = act)(y2)
    y2 = Dropout(drp)(y2)
    y2 = Dense(4, activation = eact, name = 'rotation')(y2)

    y3 = Dense(256, activation = act)(x)
    y3 = Dropout(drp)(y3)
    y3 = Dense(128, activation = act)(y3)
    y3 = Dropout(drp)(y3)
    y3 = Dense(89, activation = eact, name = 'subject')(y3)

    ########################################  DECODER  ############################################

    y4 = Conv2DTranspose(256, (3, 3), strides = (2, 2), activation = act, padding = s)(layer)
    y4 = Conv2D(256, (3, 3), activation = act, padding = s)(y4)
    y4 = Conv2D(256, (3, 3), activation = act, padding = s)(y4)
    y4 = BatchNormalization()(y4)
    y4 = Dropout(0.5)(y4)

    y4 = Conv2DTranspose(64, (3, 3), strides = (2, 2), activation = act, padding = s)(y4)
    y4 = Conv2D(64, (3, 3), activation = act, padding = s)(y4)
    y4 = Conv2D(64, (3, 3), activation = act, padding = s)(y4)
    y4 = BatchNormalization()(y4)
    y4 = Dropout(0.5)(y4)

    y4 = Conv2DTranspose(16, (3, 3), strides = (2, 2), activation = act, padding = s)(y4)
    y4 = Conv2D(16, (3, 3), activation = act, padding = s)(y4)
    y4 = Conv2D(16, (3, 3), activation = act, padding = s)(y4)
    y4 = BatchNormalization()(y4)
    y4 = Dropout(0.5)(y4)

    y4 = Conv2D(1, (1, 1), padding = s, name = 'mask')(y4)

    model = Model(inputs = model_input, outputs = [y1, y2, y3, y4])
    model.compile(loss = loss_list, optimizer = opt, metrics = test_metrics)

    return model


model = get_model()
model.summary()

## train

In [ ]:
def tr():
    chk_loss = ModelCheckpoint(filepath = 'checkpoint_loss6', monitor = 'val_loss', mode = 'min', verbose = 1, save_best_only = True)
    early_st = EarlyStopping(monitor = "val_loss", patience = 30, verbose = 1, mode = "min")
    rduce_lr = ReduceLROnPlateau(monitor = "val_subject_loss", factor = 0.5, patience = 5, verbose = 1, mode = "min", min_lr = 0.000001)
    tr_plot = PlotLossesCallback()
    lgg = tf.keras.callbacks.CSVLogger('kp_epoches.csv')
    return [chk_loss, rduce_lr, tr_plot]


cll = tr()
history = model.fit(train_gen,
                    validation_data = val_gen,
                    batch_size = bs,
                    epochs = 200,
                    verbose = 1,
                    callbacks = cll,
                    steps_per_epoch = len(os.listdir('F:/poorya/datasets/keypoint J dataset/pretext dataset/train_aug/')) // bs,
                    validation_steps = len(os.listdir('F:/poorya/datasets/keypoint J dataset/pretext dataset/val/')) // bs
                    )

## evaluation

In [ ]:
testmodel_loss = load_model('checkpoint_loss6', custom_objects = {'PSNR': PSNR, 'SSIMLoss': SSIMLoss})
testmodel_loss.evaluate(test_gen, steps = len(os.listdir('F:/poorya/datasets/keypoint J dataset/pretext dataset/val/')) // bs)

In [ ]:
#testmodel = load_model('checkpoint')
folder = 'F:/poorya/datasets/keypoint J dataset/pretext dataset/test/'
i = 1
n = os.listdir(folder)
n.sort(key = len)

images = []
mask = []
rotation = []
gender = []
subject = []

train_img_1 = image.img_to_array(image.load_img(folder + n[i], color_mode = "grayscale", target_size = (384, 384)))
train_img_2 = image.img_to_array(image.load_img(folder + n[i], color_mode = "grayscale", target_size = (384, 384)))
train_img_1 /= 255.0
train_img_2 /= 255.0

mask.append(train_img_1[168:216, 168:216])
train_img_2[168:216, 168:216] = (0)
images.append(train_img_2)

rotation.append(to_categorical(int(n[i][-10]), num_classes = 4))
subject.append(to_categorical(int(n[i][-6:-4]) - 1, num_classes = 89))
gender.append(to_categorical(int(n[i][-8]), num_classes = 2))
y1_pred, y2_pred, y3_pred, y4_pred = model.predict(np.array(images))

plt.imshow(np.array(mask)[0, :, :, :])
plt.show()
plt.imshow(y4_pred[0, :, :, :])
plt.show()